In [125]:
import os
import xmltodict
import pandas as pd
import pickle as pkl
from src.model import SoccerObjectDetector

In [129]:
det = SoccerObjectDetector(bboxpath=os.path.abspath("./bbox_train.pkl"))

In [130]:
train_dir=os.path.abspath("./notebooks/training_all/")
det(None, image_dir=train_dir)

In [131]:
cores = [f.split(".")[0] for f in os.listdir(train_dir) if "jpg" in f]
jpgs = list(map(lambda x: x + ".jpg", cores))
xmls = list(map(lambda x: os.path.join(train_dir, x + ".xml"), cores))

In [148]:
bb_df = det.bb_df
bb_df['path'] = bb_df['fileName'].apply(lambda x: x[0].replace("\\", "/"))
bb_df['img_num']= bb_df['path'].apply(lambda x: x.split("/")[-1].split(".")[0])
bb_df

,batchNum,fileName,label,left,top,width,height,path,img_num
0,0,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,397.299530,547.161743,59.360138,148.942993,C:/Users/Celso/Documents/soccer/notebooks/trai...,241
1,0,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,587.159302,369.398193,50.045349,105.433990,C:/Users/Celso/Documents/soccer/notebooks/trai...,241
2,0,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,990.826965,281.036987,53.095398,85.126892,C:/Users/Celso/Documents/soccer/notebooks/trai...,241
3,1,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,1009.728271,546.355713,52.111572,119.259277,C:/Users/Celso/Documents/soccer/notebooks/trai...,2001
4,1,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,273.231323,449.234985,53.859436,115.204041,C:/Users/Celso/Documents/soccer/notebooks/trai...,2001
...,...,...,...,...,...,...,...,...,...
805,62,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,428.231384,327.312897,22.440460,49.062958,C:/Users/Celso/Documents/soccer/notebooks/trai...,22621
806,62,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,1074.808105,225.620834,13.701904,40.894791,C:/Users/Celso/Documents/soccer/notebooks/trai...,22621
807,62,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,1121.270996,422.744415,27.847900,56.051727,C:/Users/Celso/Documents/soccer/notebooks/trai...,22621
808,62,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,257.098938,186.510071,14.220154,38.263000,C:/Users/Celso/Documents/soccer/notebooks/trai...,22621


In [160]:
df = pd.DataFrame()
dfs = []
for i, xml in enumerate(xmls):
    with open(xml, "r") as f:
        d = xmltodict.parse("".join(f.readlines()))
    a = d['annotation']
    path = a['path'].replace("\\", "/")
    data = a['object']
    tmp = pd.DataFrame(
        [{"xmin_known":di['bndbox']["xmin"],
            "ymin_known":di['bndbox']['ymin'],
            "xmax_known":di['bndbox']['xmax'],
            "ymax_known":di['bndbox']['ymax'],
            "label_known":di['name'],
            "path":path }
    for di in data])
    dfs.append(tmp.copy())

In [161]:
known = pd.concat(dfs)
known.to_pickle("known_coords.pkl")
known['img_num'] = known['path'].apply(lambda x: x.split("/")[-1].split(".")[0])

In [162]:
bb_df.to_pickle("predicted_coords.pkl")

In [163]:
bb_df.shape

(810, 9)

In [164]:
dfs

[   xmin_known ymin_known xmax_known ymax_known label_known  \
 0         193        162        229        238      player   
 1          78        326        128        436      player   
 2         303        387        358        495      player   
 3         604        403        676        518      player   
 4         709        292        753        388      player   
 5         911        247        947        340      player   
 6         963        220       1000        302      player   
 7        1154         82       1180        139      blurry   
 8        1086        357       1129        471      player   
 9         603        165        644        229      player   
 10        532        132        578        190      player   
 11        330        263        379        356      player   
 12        387        281        433        363      blurry   
 13        358        337        378        359        ball   
 14        477        309        547        402      pl

In [165]:
known[known['label_known'] == 'blurry']

,xmin_known,ymin_known,xmax_known,ymax_known,label_known,path,img_num
7,1154,82,1180,139,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801
12,387,281,433,363,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801
1,91,253,131,344,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,11041
6,308,392,366,488,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,11041
12,1159,395,1212,506,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,11041
...,...,...,...,...,...,...,...
12,230,36,257,73,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,8881
13,72,120,126,208,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,8881
15,309,236,375,337,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,8881
18,218,191,271,288,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,8881


In [166]:
pred = bb_df
pred

,batchNum,fileName,label,left,top,width,height,path,img_num
0,0,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,397.299530,547.161743,59.360138,148.942993,C:/Users/Celso/Documents/soccer/notebooks/trai...,241
1,0,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,587.159302,369.398193,50.045349,105.433990,C:/Users/Celso/Documents/soccer/notebooks/trai...,241
2,0,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,990.826965,281.036987,53.095398,85.126892,C:/Users/Celso/Documents/soccer/notebooks/trai...,241
3,1,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,1009.728271,546.355713,52.111572,119.259277,C:/Users/Celso/Documents/soccer/notebooks/trai...,2001
4,1,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,273.231323,449.234985,53.859436,115.204041,C:/Users/Celso/Documents/soccer/notebooks/trai...,2001
...,...,...,...,...,...,...,...,...,...
805,62,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,428.231384,327.312897,22.440460,49.062958,C:/Users/Celso/Documents/soccer/notebooks/trai...,22621
806,62,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,1074.808105,225.620834,13.701904,40.894791,C:/Users/Celso/Documents/soccer/notebooks/trai...,22621
807,62,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,1121.270996,422.744415,27.847900,56.051727,C:/Users/Celso/Documents/soccer/notebooks/trai...,22621
808,62,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,257.098938,186.510071,14.220154,38.263000,C:/Users/Celso/Documents/soccer/notebooks/trai...,22621


In [167]:
comb = pd.merge(known, pred, on='img_num')

In [168]:
known.shape

(953, 7)

In [169]:
bb_df.shape

(810, 9)

In [170]:
comb.to_pickle("pred_known.pkl")
comb

,xmin_known,ymin_known,xmax_known,ymax_known,label_known,path_x,img_num,batchNum,fileName,label,left,top,width,height,path_y
0,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,609.471680,405.075195,62.954163,105.912018,C:/Users/Celso/Documents/soccer/notebooks/trai...
1,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,329.556335,262.123199,43.779266,95.501587,C:/Users/Celso/Documents/soccer/notebooks/trai...
2,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,304.836792,389.360626,52.285095,111.099762,C:/Users/Celso/Documents/soccer/notebooks/trai...
3,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,822.495667,204.226395,27.109985,74.721359,C:/Users/Celso/Documents/soccer/notebooks/trai...
4,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,194.103409,165.580643,30.723877,69.476288,C:/Users/Celso/Documents/soccer/notebooks/trai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12859,877,186,910,251,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,1077.041748,466.650330,35.615845,91.188232,C:/Users/Celso/Documents/soccer/notebooks/trai...
12860,877,186,910,251,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,896.193787,464.618256,55.236572,66.171234,C:/Users/Celso/Documents/soccer/notebooks/trai...
12861,877,186,910,251,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,620.913696,268.539429,43.558411,67.601044,C:/Users/Celso/Documents/soccer/notebooks/trai...
12862,877,186,910,251,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,965.717712,411.003082,31.991943,59.050873,C:/Users/Celso/Documents/soccer/notebooks/trai...


In [174]:
for col in ['xmax_known', 'xmin_known', 'ymax_known', 'ymin_known', 'left', 'top', 'width', 'height']:
    comb[col] = comb[col].astype(float)
comb['mid_x_known'] = comb['xmin_known'] + (comb['xmax_known'] - comb['xmin_known']) / 2
comb['mid_y_known'] = comb['ymin_known'] + (comb['ymax_known'] - comb['ymin_known']) / 2
comb['mid_x_pred'] = comb['left'] + comb['width'] / 2
comb['mid_y_pred'] = comb['top'] + comb['height'] / 2
comb

,xmin_known,ymin_known,xmax_known,ymax_known,label_known,path_x,img_num,batchNum,fileName,label,left,top,width,height,path_y,mid_x_known,mid_y_known,mid_x_pred,mid_y_pred
0,193.0,162.0,229.0,238.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,609.471680,405.075195,62.954163,105.912018,C:/Users/Celso/Documents/soccer/notebooks/trai...,211.0,200.0,640.948761,458.031204
1,193.0,162.0,229.0,238.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,329.556335,262.123199,43.779266,95.501587,C:/Users/Celso/Documents/soccer/notebooks/trai...,211.0,200.0,351.445969,309.873993
2,193.0,162.0,229.0,238.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,304.836792,389.360626,52.285095,111.099762,C:/Users/Celso/Documents/soccer/notebooks/trai...,211.0,200.0,330.979340,444.910507
3,193.0,162.0,229.0,238.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,822.495667,204.226395,27.109985,74.721359,C:/Users/Celso/Documents/soccer/notebooks/trai...,211.0,200.0,836.050659,241.587074
4,193.0,162.0,229.0,238.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,194.103409,165.580643,30.723877,69.476288,C:/Users/Celso/Documents/soccer/notebooks/trai...,211.0,200.0,209.465347,200.318787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12859,877.0,186.0,910.0,251.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,1077.041748,466.650330,35.615845,91.188232,C:/Users/Celso/Documents/soccer/notebooks/trai...,893.5,218.5,1094.849670,512.244446
12860,877.0,186.0,910.0,251.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,896.193787,464.618256,55.236572,66.171234,C:/Users/Celso/Documents/soccer/notebooks/trai...,893.5,218.5,923.812073,497.703873
12861,877.0,186.0,910.0,251.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,620.913696,268.539429,43.558411,67.601044,C:/Users/Celso/Documents/soccer/notebooks/trai...,893.5,218.5,642.692902,302.339951
12862,877.0,186.0,910.0,251.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,965.717712,411.003082,31.991943,59.050873,C:/Users/Celso/Documents/soccer/notebooks/trai...,893.5,218.5,981.713684,440.528519


In [175]:
import numpy as np

In [182]:
comb['dist_mid'] = np.linalg.norm(comb[['mid_x_known', 'mid_y_known']].values - comb[['mid_x_pred', 'mid_y_pred']].values, axis=1)

In [188]:
min_dist=comb.groupby(['xmin_known', 'ymin_known', 'xmax_known', 'ymax_known']).min()

In [208]:
min_dist = comb.loc[comb.groupby(['xmin_known', 'ymin_known', 'xmax_known', 'ymax_known'])['dist_mid'].idxmin()]

In [217]:
min_dist = min_dist[min_dist['dist_mid'] > 70]

In [220]:
sum(min_dist['label_known'] == 'blurry')/sum(known['label_known'] == 'blurry')

0.23893805309734514

In [221]:
sum(min_dist['label_known'] == 'player')/sum(known['label_known'] == 'player')

0.13810741687979539

In [222]:
sum(min_dist['label_known'] == 'player')

108

In [223]:
sum(known['label_known'] == 'player')

782

In [224]:
sum(min_dist['label_known'] == 'blurry')

27

In [225]:
sum(known['label_known'] == 'blurry')

113

In [249]:
min_dist

,xmin_known,ymin_known,xmax_known,ymax_known,label_known,path_x,img_num,batchNum,fileName,label,left,top,width,height,path_y,mid_x_known,mid_y_known,mid_x_pred,mid_y_pred,dist_mid
3099,1.0,279.0,18.0,367.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,15841,53,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,214.372086,214.892197,32.054123,34.431152,C:/Users/Celso/Documents/soccer/notebooks/trai...,9.5,323.0,230.399147,232.107773,238.867809
1303,7.0,210.0,33.0,255.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,14211,43,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,350.462677,404.897461,25.099426,64.661285,C:/Users/Celso/Documents/soccer/notebooks/trai...,20.0,232.5,363.012390,437.228104,399.463510
751,11.0,459.0,65.0,544.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,12721,40,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,78.383301,382.174194,26.918633,71.280334,C:/Users/Celso/Documents/soccer/notebooks/trai...,38.0,501.5,91.842617,417.814362,99.510369
8591,12.0,425.0,68.0,544.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,5751,19,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,249.580917,294.625580,44.298508,89.011841,C:/Users/Celso/Documents/soccer/notebooks/trai...,40.0,484.5,271.730171,339.131500,273.552322
4781,15.0,247.0,50.0,334.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,241,0,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,397.299530,547.161743,59.360138,148.942993,C:/Users/Celso/Documents/soccer/notebooks/trai...,32.5,290.5,426.979599,621.633240,515.037257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8616,1219.0,216.0,1249.0,290.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,5751,19,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,969.673523,282.397736,23.621643,70.529358,C:/Users/Celso/Documents/soccer/notebooks/trai...,1234.0,253.0,981.484344,317.662415,260.663354
6848,1249.0,184.0,1280.0,229.0,blurry,C:/Users/Celso/Documents/soccer/notebooks/trai...,4081,12,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,872.509888,200.505646,33.912292,69.436584,C:/Users/Celso/Documents/soccer/notebooks/trai...,1264.5,206.5,889.466034,235.223938,376.132344
2298,1250.0,448.0,1280.0,549.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,14881,48,(C:\Users\Celso\Documents\soccer\notebooks\tra...,omega,742.060486,484.522949,35.632324,85.021301,C:/Users/Celso/Documents/soccer/notebooks/trai...,1265.0,498.5,759.876648,527.033600,505.928619
6021,1255.0,340.0,1280.0,438.0,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,3751,8,(C:\Users\Celso\Documents\soccer\notebooks\tra...,alpha,1204.786133,245.395523,31.639893,98.954056,C:/Users/Celso/Documents/soccer/notebooks/trai...,1267.5,389.0,1220.606079,294.872551,105.161858


In [234]:
df = pd.read_pickle("pred_known.pkl")
df = df[[col for col in df.columns if col not in ['fileName', 'path_y']]]
df

,xmin_known,ymin_known,xmax_known,ymax_known,label_known,path_x,img_num,batchNum,label,left,top,width,height
0,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,omega,609.471680,405.075195,62.954163,105.912018
1,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,omega,329.556335,262.123199,43.779266,95.501587
2,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,omega,304.836792,389.360626,52.285095,111.099762
3,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,omega,822.495667,204.226395,27.109985,74.721359
4,193,162,229,238,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,10801,38,omega,194.103409,165.580643,30.723877,69.476288
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12859,877,186,910,251,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,omega,1077.041748,466.650330,35.615845,91.188232
12860,877,186,910,251,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,alpha,896.193787,464.618256,55.236572,66.171234
12861,877,186,910,251,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,alpha,620.913696,268.539429,43.558411,67.601044
12862,877,186,910,251,player,C:/Users/Celso/Documents/soccer/notebooks/trai...,9571,37,alpha,965.717712,411.003082,31.991943,59.050873


In [255]:
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()
def calc_iou(s):
#     print(s)
    pred1 = np.zeros((1280, 720))
    pred2 = np.zeros((1280,720))
    
    xmk = int(s['xmin_known'])
    ymk = int(s['ymin_known'])
    xmak = int(s['xmax_known'])
    ymak = int(s['ymax_known'])
    
    pred1[ymk:ymak, xmk:xmak] = 1
    
    xmp = int(s['left'])
    ymp = int(s['top'])
    xmap = int(xmp + s['width'])
    ymap = int(ymp + s['height'])
    
    pred2[ymp:ymap, xmp:xmap] = 1
#     plt.imshow(pred1)
#     plt.show()
#     plt.imshow(pred2)
#     plt.show()
    
    intersection = np.sum(np.logical_or(pred1, pred2))
    union = np.sum(np.logical_and(pred1, pred2))
    iou = intersection / union
    return iou
df.progress_apply(calc_iou, axis=1)

c:\users\celso\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


c:\users\celso\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in long_scalars


KeyboardInterrupt: 